# Introducción a Pandas









**Ariel Rossanigo**


### Quien soy?

* Ariel Rossanigo
* Profe de Inteligencia Artificial
* Developer, Data Scientist


### Objetivos de la charla

* **10 Minutes to pandas** en 90 minutos


https://pandas.pydata.org/pandas-docs/stable/10min.html


### Agenda

* Numpy, Pandas, Jupyter. Instalación y prueba del ambiente
* Series y Dataframes
* Lectura de datos
* Indexado
* Operaciones
* Merge
* Group
* Plots
* Tips and tricks

#### Numpy

* Arrays multidimensionales implementados de forma eficiente
* Base para muchos de los paquetes científicos en Python

#### Jupyter notebook

* El notebook es un interprete interactivo en la web, pero que permite mezclar código, videos, imágenes, markdown, latex y gráficas...

* Un notebook es una sucesión de celdas, donde cada una puede ser código, texto, etc

#### Pandas (Python Data Analysis Library)

* Herramienta por defecto para trabajar con datos en Python
* Usa numpy por detrás pero propone abstracciones como ser Serie y DataFrame


### Preparando el ambiente...

#### 1. Clonar el repositorio de la charla e ir a la carpeta de esta charla

    git clone https://github.com/arielrossanigo/talks.git
    cd pandas_intro

#### 2. Instalar requerimientos

    pip install -r requirements.txt

#### 3. Abrir este notebook

    jupyter notebook introduccion_a_pandas.ipynb
  

#### 4. Ejecutar la siguiente celda

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("Funciona!")

### Estructuras básicas

* Serie: one dimensional labeled array

In [ ]:
nombres = pd.Series(['john', 'paul', 'george', 'ringo'])
nombres

* Dataframe: two dimensional labeled data structure with columns of potentially different types

In [ ]:
beatles = pd.DataFrame({    
    'nombre': nombres,
    'nacimiento': [1940, 1942, 1943, 1940]
})
beatles

### *Hands on*

* Crear un Dataframe como el anterior pero con 2 columnas extras

 * instrumento: en orden serían ('guitarra', 'bajo', 'guitarra', 'bateria')
 * permanencia: (9, 10, 10, 8)

In [ ]:
## Modificar este código
beatles = pd.DataFrame({    
    'nombre': nombres,
    'nacimiento': [1940, 1942, 1943, 1940]
})
beatles

### Leyendo datos

**Pandas** viene preparado para interactuar con varios formatos de datos, entre ellos CSV, Excel, HDF5, pickle, SQL y varios más. Algunos de los parámetros más usados de read_csv:

* filepath_or_buffer: requerido
* parse_dates: columnas a ser parseadas como dates
* date_parser: función utilizada para parsear dates
* usecols: columnas a recuperar
* dtype: tipos de datos de las columnas
* na_values: valores que son considerados NA

La lista continúa y es bastante larga...

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv

### El ejemplo...

Vamos a trabajar con datos expuestos por Organismos del Gobierno de la República Argentina, más precisamente por el Ministerio de Ciencia y Tecnología.

Vamos a usar 2 datasets:

* Proyectos de ciencia, tecnología e innovación (http://datos.gob.ar/dataset/proyectos-ciencia-tecnologia-e-innovacion)
* Empresas de ejecución de proyectos de ciencia, tecnología e innovación (http://datos.gob.ar/dataset/empresas-ejecucion-proyectos-ciencia-tecnologia-e-innovacion)

In [ ]:
proyectos = pd.read_csv('proyectos.csv')
proyectos.head(3)

In [ ]:
# Un poco de detalle
proyectos.info(memory_usage='deep')

### *Hands on*

* Leer nuevamente el csv, pero:

 * recuperar solamente las columnas: 'proyecto_id', 'fecha_inicio', 'provincia_de_ejecución', 'monto_financiado', 'monto_total', 'gran_area_conocimiento', 'tipo_organizacion_ejec'
 * la columna ``fecha_inicio`` debe ser parseada como date

In [ ]:
## Modificar este código
proyectos = pd.read_csv('proyectos.csv')
proyectos.set_index('proyecto_id', inplace=True)
proyectos.head(3)

### Filtrado de datos

* By label: **loc**
* By position: **iloc**
* Boolean indexing: *a la numpy*

In [ ]:
proyectos.loc[1:3, 'fecha_inicio':'monto_total']

In [ ]:
proyectos.iloc[1:3, 0:3]

In [ ]:
proyectos[(proyectos['provincia_de_ejecución'] == 'San Luis') & (proyectos.fecha_inicio.dt.year == 2016)]

### *Hands on*

* Mostrar los proyectos de CABA, donde se haya financiado más de 1MM de pesos.

In [ ]:
## Modificar este código
filtro = (proyectos['provincia_de_ejecución'] == 'CABA') & (proyectos.monto_financiado > 1e6)
proyectos[filtro].head(5)

### Agregado de columnas, operaciones básicas

In [ ]:
# la forma mas simple, con operaciones entre series
proyectos['porcentaje_financiado'] = proyectos.monto_financiado / proyectos.monto_total

In [ ]:
# valor condicional
proyectos['financia_mas_80_por_ciento'] = np.where(proyectos.porcentaje_financiado > 0.8, 'Si', 'No')

### Stats

In [ ]:
print("Monto de proyecto. Promedio: {:,.2f} $. Desvio: {:,.2f} $".format(
    proyectos.monto_total.mean(),
    proyectos.monto_total.std()
))

In [ ]:
proyectos.describe(include='all')

In [ ]:
# cuantos proyectos por provincia?
proyectos['provincia_de_ejecución'].value_counts().head(5)

In [ ]:
# bins de montos de proyecto (rangos fijos)
bins = [0, 1e6, 2e7, 1e20]
names = ['Barato', 'Normal', 'Caro']

proyectos['costo'] = pd.cut(proyectos.monto_total, bins, labels=names)
proyectos.costo.value_counts()

In [ ]:
bins = [0, .33, .66, 1]
proyectos['costo'] = pd.qcut(proyectos.monto_total, bins, labels=names)
proyectos.costo.value_counts()

### Aplicando funciones

In [ ]:
proyectos.monto_total.apply(lambda x: '{:,.2f} $'.format(x)).head(3)

In [ ]:
proyectos.apply(lambda x: x.monto_total - x.monto_financiado, axis='columns').head(3)

### Funciones con strings

Hay un atributo *str* para tal fin

In [ ]:
# pasar a minúsculas
proyectos.tipo_organizacion_ejec.str.lower().head(3)

# contiene universidad o ciencia
ix = proyectos.tipo_organizacion_ejec.str.contains('universidad|ciencia', case=False)
proyectos[ix].tipo_organizacion_ejec.unique()

### Algunos métodos útiles

* drop_duplicates: si hay varias filas repetidas deja solo una (no tiene en cuenta el índice)
* fillna: completa con el valor que recibe como parámetro las celdas sin valor


In [ ]:
proyectos.gran_area_conocimiento.fillna('???').unique()

In [ ]:
proyectos.gran_area_conocimiento.drop_duplicates()

### *Hands on*

* ¿Cuál es el área de conocimiento con más proyectos?
* ¿Qué porcentaje del costo se financia en promedio?

In [ ]:
# lugar para responder las preguntas


### Agrupando datos

Involucra 1 o más de los siguientes pasos:

* **Separar** los datos en grupos en base a algún criterio
* **Aplicar** una función a cada grupo de forma independiente

 * Aggregation
 * Transformation
 * Filtration

* **Combinar** los resultados en una estructura de datos

https://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby

In [ ]:
# cantidad de proyectos y promedio de monto financiado por provincia
proyectos.groupby(proyectos['provincia_de_ejecución']).monto_financiado.agg(['mean', 'count']).head(3)

In [ ]:
# los 3 proyectos con mayor financiacion por provincia
ordenado = proyectos.sort_values(by=['provincia_de_ejecución', 'monto_financiado'], ascending=False)
ordenado.groupby('provincia_de_ejecución').head(3).head(6)

### *Hands on*

En el dataset se puede apreciar que hay más de un registro por proyecto. Esto se debe a que el mismo proyecto puede estar en más de una provincia a la vez o abarcar más de un área de conocimiento.

* ¿Cuánto es el monto total financiado en cada año sabiendo lo antes mencionado? 

In [ ]:
## Completar código aquí

### Combinando datos

#### Concat

    pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
              keys=None, levels=None, names=None, verify_integrity=False,
              copy=True)


In [ ]:
pd.concat([beatles, beatles], axis=0)

### Combinando datos

#### Merge

    pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)


In [ ]:
muertes = pd.DataFrame({
    'nombre': ['john', 'george'],
    'año de muerte': [1980, 2001] 
})

pd.merge(beatles, muertes, on='nombre')

In [ ]:
m2 = muertes.set_index('nombre')

pd.merge(beatles, m2, left_on='nombre', right_index=True, how='left', indicator=True, validate='one_to_one')

In [ ]:
instrumentos = pd.DataFrame({
    'nombre': ['john', 'john', 'ringo',  'ringo', 'charly'],
    'instrumento': ['guitarra', 'teclado', 'bateria', 'percusión', 'piano'] 
})

pd.merge(beatles, instrumentos, left_on='nombre', right_on='nombre', how='outer', indicator=True, 
         validate='one_to_many')

### *Hands on*

* Usar el dataset de empresas provisto debajo para determinar el top 5 de empresas en cuanto a su involucración en los  proyectos de mayor monto

In [ ]:
empresas = pd.read_csv('empresas.csv')
empresas.head(3)

In [ ]:
## Completar código aquí

### Ploteando datos

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2015', periods=1000))
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
df.plot(figsize=(12, 4));

In [ ]:
f, axis = plt.subplots(1, 2, figsize=(12, 4))
df.boxplot(ax=axis[0])
df.A.hist(ax=axis[1]);

### *Hands on*

* ¿Cuánto es el monto total financiado año a año? 
* ¿Cuánto es el monto total financiado provincia por provincia? 

In [ ]:
f, axis = plt.subplots(1, 2, figsize=(12, 4))
## Completar código aquí


### Algunos consejos 

* Evitar lo más posible utilizar ``apply`` => Tratar de usar operaciones sobre vectores
* Evitar ``iterrows`` => Acceder 
* ``concat`` duplica el consumo de memoria al momento de la concatenación => Depende del caso, HDF5 quizás ayuda

### Preguntas?

<img src="../common/imgs/man-qmark.jpg" width="400" align="middle">


### Gracias!

Mis datos de contacto:

<p><img src="../common/imgs/gmail-1162901_960_720.png" width="40" style="float: left;" align="middle"> arielrossanigo@gmail.com</p>

<p><img src="../common/imgs/twitter-312464_960_720.png" width="40" style="float: left;" align="middle"> @arielrossanigo</p>

<p><img src="../common/imgs/github-154769__340.png" width="40" style="float: left;" align="middle"> https://github.com/arielrossanigo</p>
